<a href="https://colab.research.google.com/github/dong-yf/100-Days-Of-ML-Code/blob/master/flower_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import cv2
from PIL import Image
import torch
import matplotlib.image as mpimg
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [3]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [4]:
#parameters
BATCH_SIZE = 32

In [23]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5, 0.5, 0.5])
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5, 0.5, 0.5])
])

In [24]:
class myDataset(Dataset):
    
    def __init__(self, image, label, transform):
        self.image = image
        self.label = label
        self.transform = transform
    
    def __len__(self):
        return len(self.image)
    
    def __getitem__(self, index):
        image, label = self.image[index], int(self.label[index])
        image = Image.fromarray(np.core.uint8(image)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [25]:
data_path = 'data/train/'
img_names = os.listdir(data_path)
print(os.listdir('data/'))
x_train = []
y_train = []
for img_name in img_names:
    img = plt.imread(data_path+img_name)
    x_train.append(img)
    y_train.append(int(img_name[0]))
x_train = np.array(x_train)
y_train = np.array(y_train)

['train', 'test', 'train_X (1).npy', 'train.npz', 'train_data', 'train_X.npy']


KeyboardInterrupt: ignored

In [26]:
data = np.load('data/train.npz')
x_train, y_train = data['dataX'], data['dataY']

In [27]:
test_data_path = 'data/test/'
test_img_names = os.listdir(test_data_path)
x_test = []
y_test = []
for test_img_name in test_img_names:
    img = plt.imread(test_data_path+test_img_name)
    x_test.append(img)
    y_test.append(1)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [28]:
x_train = myDataset(x_train, y_train, train_transform)
x_test = myDataset(x_test, y_test, test_transform)

In [29]:
train_loader = DataLoader(dataset=x_train,batch_size=BATCH_SIZE,shuffle=True)
test_loader = DataLoader(dataset=x_test,batch_size=BATCH_SIZE,shuffle=False)

In [34]:
model = models.resnet50(pretrained=False).cuda()
loss = nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(),lr = 0.1)
num_epochs = 30

In [ ]:
num_epochs = 60
losses = [] 
acces = []
eval_losses = []
eval_acces = []

for echo in range(num_epochs):
    train_loss = 0   #定义训练损失
    train_acc = 0    #定义训练准确度
    model.train()    #将网络转化为训练模式
    for i,(X,label) in enumerate(train_loader):     #使用枚举函数遍历train_loader
        X = Variable(X).cuda()      #包装tensor用于自动求梯度
        label = Variable(label).cuda()
        out = model(X)            #正向传播
        lossvalue = loss(out,label)         #求损失值
        optimizer.zero_grad()       #优化器梯度归零
        lossvalue.backward()    #反向转播，刷新梯度值
        optimizer.step()        #优化器运行一步，注意optimizer搜集的是model的参数
        
        #计算损失
        train_loss += float(lossvalue)      
        #计算精确度
        _,pred = out.max(1)
        num_correct = (pred == label).sum()
        acc = int(num_correct) / X.shape[0]
        train_acc += acc
    
    losses.append(train_loss / len(train_loader))
    acces.append(train_acc / len(train_loader))
    print("echo:"+' ' +str(echo))
    print("lose:" + ' ' + str(train_loss / len(train_loader)))
    print("accuracy:" + ' '+str(train_acc / len(train_loader)))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


echo: 0
lose: 2.431580784431113
accuracy: 0.3099531938325991
echo: 1
lose: 1.2396550449637065
accuracy: 0.5412307268722467
echo: 2
lose: 1.0227613902039465
accuracy: 0.6223843612334802
echo: 3
lose: 0.8814597961529761
accuracy: 0.675454295154185
echo: 4
lose: 0.7819568790528217
accuracy: 0.7100770925110133
echo: 5
lose: 0.7076222781699134
accuracy: 0.7370594713656388
echo: 6
lose: 0.6305231982719005
accuracy: 0.7642483480176211
echo: 7
lose: 0.5632990525157441
accuracy: 0.790955396475771
echo: 8
lose: 0.5019343658165785
accuracy: 0.8166299559471366
echo: 9
lose: 0.4422588095987946
accuracy: 0.8419603524229075
echo: 10
lose: 0.37280657480466733
accuracy: 0.8641244493392071
echo: 11
lose: 0.3323969430729156
accuracy: 0.8793364537444934
echo: 12
lose: 0.28665060764137607
accuracy: 0.895443281938326
echo: 13
lose: 0.2336279351493992
accuracy: 0.915955396475771
echo: 14
lose: 0.20430392806012068
accuracy: 0.9257296255506607
echo: 15
lose: 0.19483255205823657
accuracy: 0.9321998898678414
ech